In [ ]:
!wget https://www.dropbox.com/s/241p0ooxybbkzpq/archive.zip?dl=0

In [ ]:
!unzip archive.zip\?dl\=0.1

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
%pylab inline
import matplotlib.image as mpimg
import PIL
from PIL import Image
import cv2

len(os.listdir("/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train"))

In [ ]:
!pip install Keras-Preprocessing
import keras
from keras_preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.applications.vgg19 import VGG19 , preprocess_input , decode_predictions

In [ ]:
train_datagen = ImageDataGenerator(zoom_range= 0.5 ,shear_range = 0.3 ,horizontal_flip= True,preprocessing_function = preprocess_input )
val_datagen = ImageDataGenerator(preprocessing_function = preprocess_input )

In [ ]:
train = train_datagen.flow_from_directory(directory = "/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train",target_size = (256,256),batch_size=32)

val = val_datagen.flow_from_directory(directory = "/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid",target_size = (256,256),batch_size=32)



In [ ]:
t_img,label = train.next ()

In [ ]:
t_img.shape

In [ ]:
def plotImage(img_arr , label ):

  for im , l in zip(img_arr, label):
    plt.figure (figsize=(5,5))
    plt.imshow(im)
    plt.show()


In [ ]:
plotImage(t_img[:3], label[:3])

In [ ]:
from keras.layers import Dense, Flatten
from keras.models import Model
from keras.applications.vgg19 import VGG19
import keras

In [ ]:
base_model = VGG19(input_shape = (256,256,3), include_top = False )

In [ ]:
for layer in base_model.layers:
  layer.trainable = False

In [ ]:
base_model.summary()

In [ ]:
X = Flatten()(base_model.output)
X = Dense (units = 38,activation = 'softmax')(X)

model = Model(base_model.input,X)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',loss= keras.losses.categorical_crossentropy,metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
#early stopping
es= EarlyStopping(monitor = 'val_accuracy', min_delta = 0.01 , patience = 10, verbose = 1)
#Model Checkpoint
mc = ModelCheckpoint ( filepath=" best_model.h5",
                      monitor='val_accuracy',
                      min_delta=0.01,
                      patience=10,
                      verbose=1,
                      save_best_only= True)
cb = [ es, mc]



In [ ]:
his = model.fit_generator(train ,
                          steps_per_epoch=16,
                          epochs=40,
                          verbose=1,
                          callbacks=cb,
                          validation_data=val,
                          validation_steps=16 )

In [ ]:
h= his.history
h.keys()


In [ ]:
plt.plot(h['accuracy'])
plt.plot(h['val_accuracy'], c = "red")

plt.title("acc vs v-acc")
plt.show()

In [ ]:
plt.plot(h['loss'])
plt.plot(h['val_loss'], c = "red")

plt.title("loss vs v-loss")
plt.show()

In [ ]:
from google.colab import files
model_json=model.to_json()
with open("model1.json","w") as json_file:
  json_file.write(model_json)
  #serialize weights to HDF5
  model.save_weights("my_model_weights.h5")
  files.download("my_model_weights.h5")
  model.save("best_model.h5")
  files.download("best_model.h5")
  print("saved model to disk")

In [ ]:
acc = model.evaluate_generator(val)[1]
print(f"The accuracy of your model is = {acc*100} %")

In [ ]:
ref=dict(zip(list(train.class_indices.values()), list(train.class_indices.keys()) ))

In [ ]:

def prediction(path):
  img = load_img(path,target_size = (256,256))
  i= img_to_array(img)
  cv2.imwrite('Image_from_array.jpg', i)
  im = preprocess_input(i)
  print("\ndimensions of preprocessed array : ",i.shape)
  cv2.imwrite('preprocess.jpg', im)
  img = np.expand_dims(im,axis=0)
  print("\ndimensions of expanded array : ",img.shape)
  x=model.predict(img)
  print("\n",x)
  pred =np.argmax(x)
  print("\n",pred)
  print(f"\n\nThe leaf with disease of { ref[pred] }")

In [ ]:
path = "/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train/Apple___healthy/00907d8b-6ae6-4306-bfd7-d54471981a86___RS_HL 5709_flipTB.JPG"
img = mpimg.imread(path)
imgplot = plt.imshow(img)
plt.show()
prediction(path)